In [1]:
import glob
print(glob.glob("/share/*"))

['/share/stocks_to_numpy_file.ipynb', '/share/all_stocks.npy', '/share/input.txt', '/share/us-stock-dataset', '/share/Transformers1', '/share/short_stocks.npy']


In [2]:
import sys
import os

import numpy as np
import torch
from torch import nn as nn
import glob

from t4.timeseries_transformer2 import TransformerConfig, TransformerRunner

from timeseries.csv_reader import read_and_merge_csv_files

from t3_karpathy.commons.commons import BaseTransformerConfig, min_max_scaling, extract_sliding_windows, z_scale, diff, standardize
from t3_karpathy.token_codec import TokenCodec
from t4.generic_dataloader import GenericDataloader
from timeseries.timeseries_transformer import TimeseriesDataloader

import torch
import matplotlib.pyplot as plt

In [3]:
data_loader_data=torch.tensor(np.load('/share/short_stocks.npy')).transpose(0, 1).cpu()
data_loader_data.shape

torch.Size([145, 5283])

In [4]:
data = data_loader_data

In [5]:
data = diff(data, 1)
data = standardize(data, -1)

In [6]:
windowed = extract_sliding_windows(data, 8)
windowed.shape

torch.Size([145, 5275, 8])

In [7]:
windowed[0, 1, :]

tensor([-0.0209, -0.1449,  0.0146, -0.0593, -0.0977, -0.1043,  0.0744,  0.0050])

In [8]:
flat_windowed = windowed.reshape(windowed.shape[0]* windowed.shape[1], -1)
print(flat_windowed.shape)

torch.Size([764875, 8])


In [9]:
device = 'cuda'

torch.manual_seed(1337)

config = TransformerConfig(
    input_embed=flat_windowed.shape[1], 
    precision=torch.bfloat16, 
    batch_size=128, 
    block_size=1, 
    n_embed=128, 
    n_head=4, 
    n_layer=8, 
    learning_rate=0.001,
    causal=True
)

runner = TransformerRunner(config, flat_windowed)

runner.train_iterate_n(25000)

context = torch.zeros((1, 1), dtype=torch.long, device=device)
# print(dataloader.token_codec.decode(runner.generate(context, max_new_tokens=2000)[0].tolist()))

TypeError: TransformerConfig.__init__() got an unexpected keyword argument 'causal'

In [ ]:

# Generate a sample tensor (you can replace this with your tensor)
tensor = data_loader_data.to('cpu')

# Plotting
plt.figure(figsize=(15, 7))
for i in range(tensor.shape[0]):
    plt.plot(tensor[i].numpy(), linewidth=0.5)  # Convert tensor to numpy for plotting

plt.title("145 Time Series")
plt.xlabel("Time Steps")
plt.ylabel("Value")
plt.show()


In [ ]:
import pandas as pd
import plotly.offline as pyo
pyo.init_notebook_mode(connected=True)

import plotly.express as px

# Sample data
data = {
    'Date': pd.date_range(start='2022-01-01', periods=10, freq='D'),
    'Value': [1, 3, 5, 6, 8, 10, 12, 14, 16, 18]
}

# Creating a DataFrame
df = pd.DataFrame(data)

# Plotting using Plotly
fig = px.line(df, x='Date', y='Value', title='Time Series')
fig.show()


In [ ]:
!pip install plotly